In [1]:
import TensorFrost as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import imageio
import cv2 as cv
from IPython.display import Image
from IPython.display import Video
import os

N = 1024 
M = 1024

def Bilinear(tex, x, y):
    xi, yi = tf.floor(x), tf.floor(y)
    xf, yf = x-xi, y-yi
    xi, yi = tf.int(xi), tf.int(yi)
    oxf, oyf = 1.0-xf, 1.0-yf
    return tex[xi, yi]*oxf*oyf + tex[xi+1, yi]*xf*oyf + tex[xi, yi+1]*oxf*yf + tex[xi+1, yi+1]*xf*yf

def CubicHermit(x):
    x2 = x * x
    x3 = x2 * x
    return [-0.5 * x3 + x2 - 0.5 * x, 1.5 * x3 - 2.5 * x2 + 1.0, -1.5 * x3 + 2.0 * x2 + 0.5 * x, 0.5 * x3 - 0.5 * x2]

def CubicInterp(tex, x, y):
    xi, yi = tf.floor(x), tf.floor(y)
    xf, yf = x-xi, y-yi
    xi, yi = tf.int(xi), tf.int(yi)

    wx = CubicHermit(xf)
    wy = CubicHermit(yf)

    valueY = 0
    for j in range(-1, 3):
        valueX = 0
        for i in range(-1, 3):
            valueX = valueX + tex[xi + i, yi + j] * wx[i + 1]
        valueY = valueY + valueX * wy[j + 1]
    return valueY

def EulerAdvection(vx, vy, dt):
    i,j = vx.indices
    x, y = tf.float(i), tf.float(j)
    x1, y1 = x - vx*dt, y - vy*dt
    return x1, y1

def RK4Advection(vx, vy, dt):
    i, j = vx.indices
    x, y = tf.float(i), tf.float(j)

    x1, y1 = x - vx*dt/2.0, y - vy*dt/2.0
    vx1, vy1 = Bilinear(vx, x1, y1), Bilinear(vy, x1, y1)

    x2, y2 = x - vx1*dt/2.0, y - vy1*dt/2.0
    vx2, vy2 = Bilinear(vx, x2, y2), Bilinear(vy, x2, y2)

    x3, y3 = x - vx2*dt, y - vy2*dt
    vx3, vy3 = Bilinear(vx, x3, y3), Bilinear(vy, x3, y3)

    x4, y4 = x - (vx + 2.0*vx1 + 2.0*vx2 + vx3)*dt/6.0, y - (vy + 2.0*vy1 + 2.0*vy2 + vy3)*dt/6.0
    return x4, y4

def SemiLagrange(vx, vy, pressure, density, dt):
    x1, y1 = RK4Advection(vx, vy, dt)
    #x1, y1 = EulerAdvection(vx, vy, dt)

    #vx = CubicInterp(vx, x1, y1)
    #vy = CubicInterp(vy, x1, y1)
    #pressure = CubicInterp(pressure, x1, y1)
    vx = CubicInterp(vx, x1, y1)
    vy = CubicInterp(vy, x1, y1)
    #pressure = Bilinear(pressure, x1, y1)
    density = CubicInterp(density, x1, y1)

    return [vx, vy, pressure, density]

def BFECC(vx, vy, pressure, dt):
    i, j = vx.indices
    x, y = tf.float(i), tf.float(j)
    
    # advect backwards
    x1, y1 = x - vx*dt, y - vy*dt
    vx1, vy1 = Bilinear(vx, x1, y1), Bilinear(vy, x1, y1)

    # advect forwards
    x2, y2 = x + vx*dt, y + vy*dt
    vx2, vy2 = Bilinear(vx1, x2, y2), Bilinear(vy1, x2, y2)

    # compute backwards forwards error correction
    vx = vx + (vx - vx2)*0.5
    vy = vy + (vy - vy2)*0.5

    # advect corrected backwards
    vx3, vy3 = Bilinear(vx, x1, y1), Bilinear(vy, x1, y1)

    return [vx3, vy3, pressure]

def Boundary(i, j):
    N1, M1 = i.shape
    return 1.0 - tf.float((i < 2) | (i > N1-3) | (j < 2) | (j > M1-3))

def Jacobi(pressure, div, iterations):
    i, j = pressure.indices

    edge = Boundary(i, j)

    # pressure solve
    for it in range(iterations):
        pressure = edge * (pressure[i-1, j] + pressure[i+1, j] + pressure[i, j-1] + pressure[i, j+1] - div) / 4.0

    return pressure

def Restrict(field):
    N1, M1 = field.shape
    N2, M2 = N1/2, M1/2
    i, j = tf.indices([N2, M2])
    i, j = 2*i, 2*j
    return 0.25*(field[i, j] + field[i+1, j] + field[i, j+1] + field[i+1, j+1])

def Prolong(field, orig):
    i, j = orig.indices
    i, j = i/2, j/2
    return orig + field[i, j]

def Residual(pressure, div):
    i, j = pressure.indices
    return div - (pressure[i-1, j] + pressure[i+1, j] + pressure[i, j-1] + pressure[i, j+1] - 4.0*pressure)

def VCycle(pressure, div):
    pressure = Jacobi(pressure, div, 1)

    res = Residual(pressure, div)
    res = Restrict(res)
    pressure0 = Jacobi(tf.zeros(res.shape), 4.0*res, 8)

    #Currently not working
    #res1 = Residual(pressure0, 4.0*res)
    #res1 = Restrict(res1)
    #pressure1 = Jacobi(tf.zeros(res1.shape), 16.0*res1, 8)
    #pressure0 = Prolong(pressure1, pressure0)

    pressure = Prolong(pressure0, pressure)

    pressure = Jacobi(pressure, div, 1)

    return pressure

def PressureSolve(pressure, div):
    pressure = VCycle(pressure, div)
    pressure = VCycle(pressure, div)
    return pressure

def Smoothstep(edge0, edge1, x):
    x = (x - edge0) / (edge1 - edge0)
    x = tf.clamp(x, 0.0, 1.0)
    return x * x * (3.0 - 2.0 * x)
    
def FluidTest():
    vx = tf.input([N, M], tf.float32)
    vy = tf.input([N, M], tf.float32)
    pressure = tf.input([N, M], tf.float32)
    density = tf.input([N, M], tf.float32)

    dt = 1.0
    i,j = vx.indices
    x, y = tf.float(i), tf.float(j)

    vx, vy, pressure, density = SemiLagrange(vx, vy, pressure, density, dt)
    
    # add source
    source = 0.16*tf.exp(-((y-M/5.0)**2.0 + (x-2.0*N/3.0)**2.0)/100.0)
    source = source - 0.15*tf.exp(-((y-4.0*M/5.0)**2.0 + (x-N/3.0)**2.0)/100.0)
    vy = vy + source
    density = density + source*source

    edge = Boundary(i, j)
    vx = vx * edge
    vy = vy * edge
    density = tf.clamp(density * edge, 0.0, 5.0)

    # pressure solve
    # compute divergence
    div = (vx[i+1, j] - vx[i-1, j] + vy[i, j+1] - vy[i, j-1]) / 2.0
    curl = tf.abs(vy[i+1, j] - vy[i-1, j] - vx[i, j+1] + vx[i, j-1]) / 2.0

    pressure = PressureSolve(pressure, div)

    # subtract pressure gradient
    gradx = (pressure[i+1, j] - pressure[i-1, j])*1.0
    grady = (pressure[i, j+1] - pressure[i, j-1])*1.0
    vx = vx - gradx
    vy = vy - grady

    # vortex confinement

    # compute gradient of curl magnitude
    gradx = (curl[i+1, j] - curl[i-1, j])*1.0
    grady = (curl[i, j+1] - curl[i, j-1])*1.0

    # normalize gradient
    mag = tf.sqrt(gradx*gradx + grady*grady) + 1e-5
    gradx = gradx / mag
    grady = grady / mag

    # compute vortex force
    vortx = -grady * curl
    vorty = gradx * curl

    # add vortex force
    vort_scale = 0.0001
    vx = vx + vortx * dt * vort_scale
    vy = vy + vorty * dt * vort_scale

    mag = 0.2*tf.sqrt(vx*vx + vy*vy)
    #mag = density

    r, g, b = 255.0*Smoothstep(0.0, 0.33, mag), 255.0*Smoothstep(0.33, 0.66, mag), 255.0*Smoothstep(0.66, 1.0, mag)

    return [vx, vy, pressure, r, g, b, div, density, Residual(pressure, div)]


tf.initialize(tf.cpu, "/O2 /fp:fast /openmp:experimental")
fluid = tf.program(FluidTest)

TensorProgram:
  Kernel count: 28
  Intermediate buffers: 39
  Lines of generated code: 771
  IR size: 4527



In [ ]:
VX = tf.memory(np.zeros((N, M)))
VY = tf.memory(np.zeros((N, M)))
PRESSURE = tf.memory(np.zeros((N, M)))
DENSITY = tf.memory(np.zeros((N, M)))

In [ ]:
#do a few steps and measure performance by timing every 100 steps
start = time.time()

#file_path = 'H:/TestVideos/fluid.gif'
#writer = imageio.get_writer(file_path, fps=30, loop=0)

#use opencv to write is as video too
video_path = 'H:/TestVideos/fluid.mp4'
fourcc = cv.VideoWriter_fourcc(*'H264')
video = cv.VideoWriter(video_path, fourcc, 60, (M, N))

for i in range(5000):
    VX, VY, PRESSURE, R, G, B, DIV, DENSITY, RES = fluid(VX, VY, PRESSURE, DENSITY)

    if i % 2 == 0:
        color = np.stack((B.numpy, G.numpy, R.numpy), axis=2)
        color_rgb = color.astype(np.uint8)
        #writer.append_data(color_rgb)
        video.write(color_rgb)

    if i % 30 == 9:
        print("Iteration: " + str(i+1) + ", IPS: " + str(30.0/(time.time()-start)))
        start = time.time()

print("Used memory: " + str(tf.used_memory()*4.0/1024.0/1024.0) + " MB")

#writer.close()
video.release()

In [ ]:
# plot color_rgb
Rnp = R.numpy
Gnp = G.numpy
Bnp = B.numpy
color = np.stack((Rnp, Gnp, Bnp), axis=-1)
color_rgb = color.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.imshow(color_rgb)
plt.colorbar()
plt.show()


In [ ]:
# plot the final velocity with colorbar
plt.figure(figsize=(10,5))
plt.imshow(np.sqrt(VX.numpy**2.0 + VY.numpy**2.0))
plt.colorbar()
plt.title('Velocity')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# plot the final pressure with colorbar
plt.figure(figsize=(10,5))
plt.imshow(PRESSURE.numpy)
plt.colorbar()
plt.title('Pressure')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# plot the final divergence with colorbar
plt.figure(figsize=(10,5))
plt.imshow(DIV.numpy)
plt.colorbar()
plt.title('Divergence')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# plot the abs residual with colorbar in log scale
plt.figure(figsize=(10,5))
plt.imshow(np.log10(np.abs(RES.numpy)), cmap='gist_rainbow')
plt.colorbar()
plt.title('Residual')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# export velocity as a png image 
from PIL import Image

Rnp = R.numpy
Gnp = G.numpy
Bnp = B.numpy
color = 0.1*np.stack((Rnp, Gnp, Bnp), axis=-1)
color_rgb = color.astype(np.uint8)

img = Image.fromarray(color_rgb, 'RGB')

img.save('H:/TestVideos/fluid_velocity.png')

